## Hamoye ML Project Electric Cars - Team Prophet v3.1

## This version 3.1 is based on following points:
#Replace fast-charge null with numeical value zero, as few cars don't support fast-charge.
#Keep Outliers in price germany
#German null prices get from pound and netherland columns.
#Train Test Split Random state=101, test_size= 0.3
#One Hot Encoding for Vehicle Names
#Drop Seats column from features, as it doesnt have linear relation with price.
#Try Random Forest with estimators=300 400 500 600 700

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('D:\\People\\Amit\\Hamoye\\project\\electric_cars_with-outlier.csv')

In [3]:
df.isnull().sum()

vehicle_name         0
model                0
battery              0
seats                0
acceleration         0
top_speed            0
distance             0
efficiency           0
fast_charge          5
price_pounds        45
price_germany       13
price_netherland    25
dtype: int64

In [4]:
# Considering the fact that five vehicles dont have Fast Charge we will replace the string - with zero
df['fast_charge'] = df['fast_charge'].replace(to_replace=np.nan, value=0)

In [5]:
df.dtypes

vehicle_name         object
model                object
battery             float64
seats                 int64
acceleration        float64
top_speed             int64
distance              int64
efficiency            int64
fast_charge         float64
price_pounds        float64
price_germany       float64
price_netherland    float64
dtype: object

In [6]:
df['fast_charge'].isnull().sum()

0

In [7]:
## Filling Null Values in germany using pounds and netherland
for i in range(len(df)):
    if (df.isna()['price_germany'][i]):
        if (df['price_pounds'][i]!=0) and (not df.isna()['price_pounds'][i]):
            df['price_germany'][i] = round(df['price_pounds'][i]/1.17)
        elif (df['price_netherland'][i]!=0) and (not df.isna()['price_netherland'][i]):
             df['price_germany'][i]= round(df['price_netherland'][i])
    
df.isnull().sum()

<ipython-input-7-faf00be62bd2>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['price_germany'][i] = round(df['price_pounds'][i]/1.17)
<ipython-input-7-faf00be62bd2>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['price_germany'][i]= round(df['price_netherland'][i])


vehicle_name         0
model                0
battery              0
seats                0
acceleration         0
top_speed            0
distance             0
efficiency           0
fast_charge          0
price_pounds        45
price_germany        0
price_netherland    25
dtype: int64

In [8]:
df.dtypes

vehicle_name         object
model                object
battery             float64
seats                 int64
acceleration        float64
top_speed             int64
distance              int64
efficiency            int64
fast_charge         float64
price_pounds        float64
price_germany       float64
price_netherland    float64
dtype: object

In [9]:
#### Now we use price_germany to get prices in USD
df['price_usd'] = round((df['price_germany']/1.18))

In [10]:
df.head()

,vehicle_name,model,battery,seats,acceleration,top_speed,distance,efficiency,fast_charge,price_pounds,price_germany,price_netherland,price_usd
0,Tesla,Model 3 Standard Range Plus LFP,52.5,5,5.6,225,350,150,560.0,40990.0,43560.0,49990.0,36915.0
1,Mercedes,EQS 580 4MATIC,107.8,5,4.3,210,610,177,800.0,115000.0,135529.0,140000.0,114855.0
2,Tesla,Model Y Long Range Dual Motor,70.0,7,5.0,217,410,171,520.0,54000.0,59965.0,65010.0,50818.0
3,Kia,EV6 GT,77.4,5,3.5,260,395,196,920.0,58295.0,65990.0,63595.0,55924.0
4,Tesla,Model 3 Long Range Dual Motor,70.0,5,4.4,233,455,154,570.0,48490.0,41444.0,NaN,35122.0


In [11]:
## Replacing categorical variable Vehicle brand name to numerical values using One Hot Encoding
df_encoded = pd.get_dummies(df, columns=['vehicle_name'])

In [12]:
df_encoded.head()

,model,battery,seats,acceleration,top_speed,distance,efficiency,fast_charge,price_pounds,price_germany,...,vehicle_name_Renault,vehicle_name_SEAT,vehicle_name_Seres,vehicle_name_Skoda,vehicle_name_Smart,vehicle_name_Sono,vehicle_name_Tesla,vehicle_name_Toyota,vehicle_name_Volkswagen,vehicle_name_Volvo
0,Model 3 Standard Range Plus LFP,52.5,5,5.6,225,350,150,560.0,40990.0,43560.0,...,0,0,0,0,0,0,1,0,0,0
1,EQS 580 4MATIC,107.8,5,4.3,210,610,177,800.0,115000.0,135529.0,...,0,0,0,0,0,0,0,0,0,0
2,Model Y Long Range Dual Motor,70.0,7,5.0,217,410,171,520.0,54000.0,59965.0,...,0,0,0,0,0,0,1,0,0,0
3,EV6 GT,77.4,5,3.5,260,395,196,920.0,58295.0,65990.0,...,0,0,0,0,0,0,0,0,0,0
4,Model 3 Long Range Dual Motor,70.0,5,4.4,233,455,154,570.0,48490.0,41444.0,...,0,0,0,0,0,0,1,0,0,0


## Features and Target

In [13]:
# In this version we decided to drop Seats and use vehicle names with coding using pd.get_dummies in Features
features = df_encoded.drop(['model','seats','price_pounds','price_germany','price_netherland','price_usd'],axis=1)
y = df['price_usd']

In [14]:
features.head()

,battery,acceleration,top_speed,distance,efficiency,fast_charge,vehicle_name_Aiways,vehicle_name_Audi,vehicle_name_BMW,vehicle_name_Byton,...,vehicle_name_Renault,vehicle_name_SEAT,vehicle_name_Seres,vehicle_name_Skoda,vehicle_name_Smart,vehicle_name_Sono,vehicle_name_Tesla,vehicle_name_Toyota,vehicle_name_Volkswagen,vehicle_name_Volvo
0,52.5,5.6,225,350,150,560.0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,107.8,4.3,210,610,177,800.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,70.0,5.0,217,410,171,520.0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,77.4,3.5,260,395,196,920.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,70.0,4.4,233,455,154,570.0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [15]:
features.tail()

,battery,acceleration,top_speed,distance,efficiency,fast_charge,vehicle_name_Aiways,vehicle_name_Audi,vehicle_name_BMW,vehicle_name_Byton,...,vehicle_name_Renault,vehicle_name_SEAT,vehicle_name_Seres,vehicle_name_Skoda,vehicle_name_Smart,vehicle_name_Sono,vehicle_name_Tesla,vehicle_name_Toyota,vehicle_name_Volkswagen,vehicle_name_Volvo
176,45.0,12.1,130,185,243,250.0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
177,45.0,12.1,130,185,243,250.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
178,45.0,12.1,130,180,250,240.0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
179,45.0,12.1,130,175,257,230.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
180,45.0,12.1,130,180,250,240.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
y.head()

0     36915.0
1    114855.0
2     50818.0
3     55924.0
4     35122.0
Name: price_usd, dtype: float64

## Normalizing the data using MinMax Scaler

In [17]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
x = pd.DataFrame(scaler.fit_transform(features), columns=features.columns)

In [18]:
x.head()

,battery,acceleration,top_speed,distance,efficiency,fast_charge,vehicle_name_Aiways,vehicle_name_Audi,vehicle_name_BMW,vehicle_name_Byton,...,vehicle_name_Renault,vehicle_name_SEAT,vehicle_name_Seres,vehicle_name_Skoda,vehicle_name_Smart,vehicle_name_Sono,vehicle_name_Tesla,vehicle_name_Toyota,vehicle_name_Volkswagen,vehicle_name_Volvo
0,0.195308,0.172414,0.355401,0.291429,0.259887,0.397163,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.496999,0.108374,0.303136,0.588571,0.412429,0.567376,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.290780,0.142857,0.327526,0.360000,0.378531,0.368794,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.331151,0.068966,0.477352,0.342857,0.519774,0.652482,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.290780,0.113300,0.383275,0.411429,0.282486,0.404255,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [19]:
x.tail()

,battery,acceleration,top_speed,distance,efficiency,fast_charge,vehicle_name_Aiways,vehicle_name_Audi,vehicle_name_BMW,vehicle_name_Byton,...,vehicle_name_Renault,vehicle_name_SEAT,vehicle_name_Seres,vehicle_name_Skoda,vehicle_name_Smart,vehicle_name_Sono,vehicle_name_Tesla,vehicle_name_Toyota,vehicle_name_Volkswagen,vehicle_name_Volvo
176,0.154392,0.492611,0.02439,0.102857,0.785311,0.177305,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
177,0.154392,0.492611,0.02439,0.102857,0.785311,0.177305,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
178,0.154392,0.492611,0.02439,0.097143,0.824859,0.170213,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
179,0.154392,0.492611,0.02439,0.091429,0.864407,0.163121,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
180,0.154392,0.492611,0.02439,0.097143,0.824859,0.170213,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Train Test Split


In [20]:
## Train Test split
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=101)

## Linear Regression


In [21]:
# Import Linear Regression Model
from sklearn.linear_model import LinearRegression
linear_model = LinearRegression()
#fit the model to the training dataset
linear_model.fit(x_train, y_train)
#obtain predictions
pred_1 = linear_model.predict(x_test)

In [22]:
# Import Metrics for evaluation
from sklearn import metrics

In [23]:
print('Mean Absolute Error (MAE)', metrics.mean_absolute_error(y_test,pred_1))
print('Mean Square Error (MSE)', metrics.mean_squared_error(y_test,pred_1))
print('Root Mean Square Error (RMSE)', np.sqrt(metrics.mean_squared_error(y_test,pred_1)))
print('R2 score', metrics.r2_score(y_test,pred_1))
print('Explained Variance score', metrics.explained_variance_score(y_test,pred_1))

Mean Absolute Error (MAE) 9364.946556851934
Mean Square Error (MSE) 183513709.98293775
Root Mean Square Error (RMSE) 13546.723219396554
R2 score 0.8213357958325704
Explained Variance score 0.8282151478829494


## Random Forest with Estimators 300

In [24]:
#Import Random Forest Estimators 300
from sklearn.ensemble import RandomForestRegressor
rfc_300 = RandomForestRegressor(n_estimators=300)
rfc_300.fit(x_train,y_train)
pred_2 = rfc_300.predict(x_test)

In [25]:
print('Mean Absolute Error (MAE)', metrics.mean_absolute_error(y_test,pred_2))
print('Mean Square Error (MSE)', metrics.mean_squared_error(y_test,pred_2))
print('Root Mean Square Error (RMSE)', np.sqrt(metrics.mean_squared_error(y_test,pred_2)))
print('R2 score', metrics.r2_score(y_test,pred_2))
print('Explained Variance score', metrics.explained_variance_score(y_test,pred_2))

Mean Absolute Error (MAE) 9656.324419985569
Mean Square Error (MSE) 286920187.20362186
Root Mean Square Error (RMSE) 16938.71858210124
R2 score 0.7206619226919277
Explained Variance score 0.726381423512964


## Random Forest with Estimators 400

In [26]:
rfc_400 = RandomForestRegressor(n_estimators=400)
rfc_400.fit(x_train,y_train)
pred_3 = rfc_400.predict(x_test)

In [27]:
print('Mean Absolute Error (MAE)', metrics.mean_absolute_error(y_test,pred_3))
print('Mean Square Error (MSE)', metrics.mean_squared_error(y_test,pred_3))
print('Root Mean Square Error (RMSE)', np.sqrt(metrics.mean_squared_error(y_test,pred_3)))
print('R2 score', metrics.r2_score(y_test,pred_3))
print('Explained Variance score', metrics.explained_variance_score(y_test,pred_3))

Mean Absolute Error (MAE) 9681.998314772727
Mean Square Error (MSE) 293072800.9326223
Root Mean Square Error (RMSE) 17119.369174494204
R2 score 0.7146718970118645
Explained Variance score 0.7218255239812039


## Random Forest with Estimators 500

In [28]:
rfc_500 = RandomForestRegressor(n_estimators=500)
rfc_500.fit(x_train,y_train)
pred_4 = rfc_500.predict(x_test)

In [29]:
print('Mean Absolute Error (MAE)', metrics.mean_absolute_error(y_test,pred_4))
print('Mean Square Error (MSE)', metrics.mean_squared_error(y_test,pred_4))
print('Root Mean Square Error (RMSE)', np.sqrt(metrics.mean_squared_error(y_test,pred_4)))
print('R2 score', metrics.r2_score(y_test,pred_4))
print('Explained Variance score', metrics.explained_variance_score(y_test,pred_4))

Mean Absolute Error (MAE) 9802.34194978355
Mean Square Error (MSE) 290061220.9735119
Root Mean Square Error (RMSE) 17031.18378074501
R2 score 0.7176038934100142
Explained Variance score 0.7232944445110936


## Random Forest with Estimators 600

In [30]:
rfc_600 = RandomForestRegressor(n_estimators=600)
rfc_600.fit(x_train,y_train)
pred_5 = rfc_600.predict(x_test)

In [31]:
print('Mean Absolute Error (MAE)', metrics.mean_absolute_error(y_test,pred_5))
print('Mean Square Error (MSE)', metrics.mean_squared_error(y_test,pred_5))
print('Root Mean Square Error (RMSE)', np.sqrt(metrics.mean_squared_error(y_test,pred_5)))
print('R2 score', metrics.r2_score(y_test,pred_5))
print('Explained Variance score', metrics.explained_variance_score(y_test,pred_5))

Mean Absolute Error (MAE) 9715.969590584413
Mean Square Error (MSE) 292963537.12119794
Root Mean Square Error (RMSE) 17116.17764342255
R2 score 0.7147782734341724
Explained Variance score 0.7213495695093264


## Random Forest with Estimators 700

In [32]:
rfc_700 = RandomForestRegressor(n_estimators=700)
rfc_700.fit(x_train,y_train)
pred_6 = rfc_700.predict(x_test)

In [33]:
print('Mean Absolute Error (MAE)', metrics.mean_absolute_error(y_test,pred_6))
print('Mean Square Error (MSE)', metrics.mean_squared_error(y_test,pred_6))
print('Root Mean Square Error (RMSE)', np.sqrt(metrics.mean_squared_error(y_test,pred_6)))
print('R2 score', metrics.r2_score(y_test,pred_6))
print('Explained Variance score', metrics.explained_variance_score(y_test,pred_6))

Mean Absolute Error (MAE) 9737.037477520098
Mean Square Error (MSE) 284930169.9091563
Root Mean Square Error (RMSE) 16879.874700635555
R2 score 0.7225993520874108
Explained Variance score 0.7269786859769084


## Random Forest with Estimators 800

In [35]:
rfc_800 = RandomForestRegressor(n_estimators=800)
rfc_800.fit(x_train,y_train)
pred_7 = rfc_800.predict(x_test)

In [36]:
print('Mean Absolute Error (MAE)', metrics.mean_absolute_error(y_test,pred_7))
print('Mean Square Error (MSE)', metrics.mean_squared_error(y_test,pred_7))
print('Root Mean Square Error (RMSE)', np.sqrt(metrics.mean_squared_error(y_test,pred_7)))
print('R2 score', metrics.r2_score(y_test,pred_7))
print('Explained Variance score', metrics.explained_variance_score(y_test,pred_7))

Mean Absolute Error (MAE) 9576.622790692642
Mean Square Error (MSE) 279990389.0889317
Root Mean Square Error (RMSE) 16732.913347320355
R2 score 0.7274085949995018
Explained Variance score 0.7324636268252022


## Random Forest with Estimators 900

In [38]:
rfc_900 = RandomForestRegressor(n_estimators=900)
rfc_900.fit(x_train,y_train)
pred_8 = rfc_900.predict(x_test)

In [39]:
print('Mean Absolute Error (MAE)', metrics.mean_absolute_error(y_test,pred_8))
print('Mean Square Error (MSE)', metrics.mean_squared_error(y_test,pred_8))
print('Root Mean Square Error (RMSE)', np.sqrt(metrics.mean_squared_error(y_test,pred_8)))
print('R2 score', metrics.r2_score(y_test,pred_8))
print('Explained Variance score', metrics.explained_variance_score(y_test,pred_8))

Mean Absolute Error (MAE) 9715.93712198172
Mean Square Error (MSE) 288684878.50475836
Root Mean Square Error (RMSE) 16990.729192849798
R2 score 0.7189438648588207
Explained Variance score 0.7244537960028858


## Xgboost Model

In [40]:
## Xgboost#Xgboost Model
from xgboost import XGBRegressor
xgb = XGBRegressor(n_estimators=1000, max_depth=7, eta=0.1, subsample=0.7, colsample_bytree=0.8)
xgb.fit(x_train,y_train)
pred_9 = xgb.predict(x_test)

In [41]:
print('Mean Absolute Error (MAE)', metrics.mean_absolute_error(y_test,pred_9))
print('Mean Square Error (MSE)', metrics.mean_squared_error(y_test,pred_9))
print('Root Mean Square Error (RMSE)', np.sqrt(metrics.mean_squared_error(y_test,pred_9))) #RMSE
print('R2 score', metrics.r2_score(y_test,pred_9)) #R2 Score
print('Explained Variance score', metrics.explained_variance_score(y_test,pred_9))

Mean Absolute Error (MAE) 10052.978053977273
Mean Square Error (MSE) 296087613.078799
Root Mean Square Error (RMSE) 17207.19654908373
R2 score 0.71173675383994
Explained Variance score 0.7132799421553342
